In [3]:
# !pip install -U qdrant-client fastembed datasets -q

In [4]:
import os
import pandas as pd
from datasets import load_dataset
from qdrant_client import QdrantClient

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/nirantk/.cache/huggingface/token
Login successful


In [6]:
dataset = load_dataset("nirantk/ecommerce-icecat-dataset", split="train")
collection_name = 'ecommerce-icecat-dataset'

In [7]:
qdrant_client = QdrantClient(
    os.environ["QDRANT_API_URL"],
    api_key = os.environ["QDRANT_API_KEY"],
)

In [8]:
names = [c.name for c in qdrant_client.get_collections().collections]
names

['ecommerce-icecat-dataset',
 'gemini-embedding-001',
 'google-palm-001',
 'google-palm-001-binary-quantization']

In [9]:
if collection_name in names:
    qdrant_client.delete_collection(collection_name)

In [10]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='gemini-embedding-001'), CollectionDescription(name='google-palm-001'), CollectionDescription(name='google-palm-001-binary-quantization')])

In [11]:
dataset

Dataset({
    features: ['id', 'name', 'title', 'ean', 'short_description', 'img_high', 'img_low', 'img_500x500', 'img_thumb', 'date_released', 'supplier', 'price'],
    num_rows: 19406
})

In [12]:
def make_document(example):
    return {
        'document': f"Name: {example['name']}\nTitle: {example['title']}\nDescription:{example['short_description']}",
    }

new_dataset = dataset.map(lambda example: make_document(example))

In [13]:
new_dataset

Dataset({
    features: ['id', 'name', 'title', 'ean', 'short_description', 'img_high', 'img_low', 'img_500x500', 'img_thumb', 'date_released', 'supplier', 'price', 'document'],
    num_rows: 19406
})

In [14]:
sample_dataset = new_dataset.select(range(1000))
text_documents, ids = sample_dataset['document'], sample_dataset['id']
ds = sample_dataset.remove_columns(['document', 'id', 'ean', 'price'])
metadata = ds.to_pandas().to_dict(orient='records')

In [15]:
# metadata

In [16]:
isinstance(metadata, dict)

False

In [17]:
qdrant_client.add(
    collection_name=collection_name,
    documents= text_documents,
    ids=ids,
    metadata = metadata
)

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Format error in JSON body: data did not match any variant of untagged enum PointInsertOperations"},"time":0.0}'